In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [22]:


url = "https://www.sec.gov/cgi-bin/srch-edgar?text=FORM-TYPE=10-k+and+FILING-DATE=202102*"

response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')

rows = soup.findAll('tr')

new_rows = rows[12:93]

filing_date = []
company_name = []
form_url = []

form = []


print(len(rows))
print(len(new_rows))

for row in new_rows:
    data = row.findAll('td')

    filingDate = str(data[4]).split(">")[1].split('<')[0]

    filing_date.append(filingDate)


    for company in data[1:2]:
            results = company.findAll('a', href=True)
            companyName = str(results).split(">")[1].split('<')[0]

            #print(companyName)
            
            company_name.append(companyName)

    for link in data[2:3]:
        new_url = link.findAll('a', href=True)[1]

        new_url = "https://www.sec.gov" + str(new_url.get('href'))

        form_url.append(new_url)

#To scrape the second page for the 10-k link, then put the links into another list
for link in range(0, len(form_url)):
    url = str(form_url[link])

    HEADERS = ({'User-Agent':
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})

    response = requests.get(url, headers = HEADERS)

    soup = BeautifulSoup(response.text, 'html.parser')

    new_url = soup.findAll('a', href=True)[9]

    try:
        form.append("https://www.sec.gov" + str(new_url.get('href').split('=')[1]))
    except:
        form.append("https://www.sec.gov" + str(new_url.get('href')))


values = {'Company Name': company_name, 'Filing Date': filing_date, '10-k Links': form}

df = pd.DataFrame(values).to_csv('formdata.csv', mode='w', index=False)

print(len(form))
print(len(company_name))


94
81
81
81
